In [5]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /home/nbuser/anaconda2_20/lib/python2.7/site-packages
Requirement already up-to-date: numpy>=1.8.2 in /home/nbuser/anaconda2_20/lib/python2.7/site-packages (from scikit-learn)
Requirement already up-to-date: scipy>=0.13.3 in /home/nbuser/anaconda2_20/lib/python2.7/site-packages (from scikit-learn)
You are using pip version 9.0.3, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics


dataset_url = "breast-cancer-data.csv"
df = pd.read_csv(dataset_url)

df.info()

ImportError: No module named model_selection

In [ ]:
df['diagnosis']=df['diagnosis'].map({'M':1,'B':0})

train, test = train_test_split(df, test_size = 0.2)

features = ['texture_mean','perimeter_mean','smoothness_mean','compactness_mean','symmetry_mean']

train_X = train[features]
train_y=train.diagnosis

test_X= test[features]
test_y =test.diagnosis

In [3]:
model=RandomForestClassifier(n_estimators=100, n_jobs=-1)
model.fit(train_X,train_y)

prediction=model.predict(test_X)
metrics.accuracy_score(prediction,test_y)

0.9210526315789473

In [4]:
from azureml import services
import numpy as np

workspace_id = "4a2b463a61124c688ea350d8913e59e5"
workspace_token = "GpiArjAg/lAsY0m+dnoPVTaBCi8KnycJb/IgzEBxQeIRP9iJCI0tQmE3InMNrMaWHMjSrqGXMgCYLvtTk5kDtg=="

@services.publish(workspace_id, workspace_token)
@services.types(texture_mean=float, perimeter_mean=float, smoothness_mean=float, compactness_mean=float, symmetry_mean=float)
@services.returns(int)
def breast_cancer_predictor(texture_mean, perimeter_mean, smoothness_mean, compactness_mean, symmetry_mean):
    return model.predict(np.array([[texture_mean, perimeter_mean, smoothness_mean, compactness_mean, symmetry_mean]]))

service_url = breast_cancer_predictor.service.url 
api_key = breast_cancer_predictor.service.api_key
help_url = breast_cancer_predictor.service.help_url
service_id = breast_cancer_predictor.service.service_id


In [1]:
breast_cancer_predictor.service(016.14, 74.00, 0.01968, 0.05914, 0.1619)

NameError: name 'breast_cancer_predictor' is not defined

In [6]:
import urllib2
import json

data = {"Inputs": {"input1": { "ColumnNames": [ "texture_mean", "perimeter_mean", "smoothness_mean", "compactness_mean", "symmetry_mean"],
                               "Values": [["018.14", "75.00", "0.09968", "0.05914", "0.1619"]]}}, # specified feature values
        "GlobalParameters": {} }
body = json.dumps(data)
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
req = urllib2.Request(service_url, body, headers) 
try:
    response = urllib2.urlopen(req)
    result = json.loads(response.read())  
    print(result['Results']['output1']['value']['Values'][0][0]) # get the returned prediction
except urllib2.HTTPError, error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(json.loads(error.read())) 

NameError: name 'api_key' is not defined